# Labelling

We can use an algorithm that generates a mask for each image downloaded referenced in the csv file.

In [6]:
from generate_mask import process_db

data_path = "data/sentinel_2"

process_db(images_path=data_path,
           output_dir=data_path,
           upsample_factor=1,
           output_type="vector"
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Processing tiles from data/sentinel_2
Processing tiles from data/sentinel_2


Alternatively, we can use scaneo to label the data by hand. In our case,  we use it just to check the masks that were generated and manually correct them if needed. 

In [2]:
!scaneo --data data/sentinel_2

Environment file .env not found.
Running command: IMAGE=vector DATA=data/sentinel_2 uvicorn api:app --port 5082 --host localhost  --app-dir /home/pcoquard/.local/lib/python3.8/site-packages/scaneo
/home/pcoquard/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings.warn("Can't initialize NVML")
Database already exists
INFO:     Started server process [744307]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://localhost:5082 (Press CTRL+C to quit)
error images:retrieve_image_tile data/sentinel_2/ship_211317180_2022-08-12.tif: No such file or directory
INFO:     127.0.0.1:45384 - "GET /images/ship_211317180_2022-08-12.tif/15/17313/10460.png?bands=4,3,2&stretch=0,3000&palette=viridis HTTP/1.1" 500 Internal Server Error
error images:retrieve_image_tile data/sentinel_2/ship_211317180_2022-08-12.tif: No such file or directoryerror images:retrieve_image_tile
 data/sentine

We can finaly clean up the database by finaly removing all the images and corresponding metadata for which the labeling is unsuccessful. 

In [15]:
import json
from pathlib import Path

for mask_path in Path(data_path).glob("*geojson"):
    with open(mask_path) as src:
        mask_not_found = json.load(src)['features'] == []

        if mask_not_found:
            image_id = mask_path.name[:-15]
            for file_path in Path(data_path).glob(f"{image_id}*"):
                # Erase the files
                file_path.unlink()

